### Make polygon class lazy load values

In [1]:
# refactor the class so calculated properties are lazy
# and shouldn't be recalculated more than once

import math


class Polygon:
    """
    Implememnt strictly convex polygon. All sides have equal len and all angles are less than 180 deg
    """
    def __init__(self, n: int, r: float):
        """
        n: int - number of edges / vertices
        r: circumradious 
        """
        if n <= 2 or r <= 0:
            raise ValueError(
                f"{type(self).__name__} expects edges / vertices and circumradious values to be positive. Number of edges / vertices must be greater than 2."
            )
        self._n = n
        self._r = r

        # refactor:
        self._interior_angle = None
        self._edge_length = None
        self._apothem = None
        self._area = None
        self._perimeter = None

    @property
    def edges(self):
        return self._n

    @property
    def vertices(self):
        return self._n

    @property
    def circumradious(self):
        return self._r
    
    @property
    def interior_angle(self):
        """
        Interior angle = (n - 2) * 180 / n
        """
        if self._interior_angle is None:
            self._interior_angle = (self.edges - 2) * (180 / self.edges)
        return self._interior_angle

    @property
    def edge_length(self):
        """
        edge length: s = 2rsin(pi / n)
        """
        if self._edge_length is None:
            self._edge_length = 2 * self.circumradious * math.sin(math.pi / self.edges)
        return self._edge_length

    @property
    def apothem(self):
        """
        apothem a = rcos(pi / n)
        """
        if self._apothem is None:
            self._apothem = self.circumradious * math.cos(math.pi / self.edges)
        return self._apothem

    @property
    def area(self):
        """
        area = 0.5 * edges_count * edge_length * apothem
        """
        if self._area is None:
            self._area = 0.5 * self.edges * self.edge_length * self.apothem
        return self._area

    @property
    def perimeter(self):
        """
        perimater = n * s
        """
        if self._perimeter is None:
            self._perimeter = self.edges * self.edge_length
        return self._perimeter
        
    def __repr__(self):
        return f"{type(self).__name__}({self.edges},{self.circumradious})"

    def __eq__(self, other: "Polygon") -> bool:
        return self.edges == other.edges and self.circumradious == other.circumradious

    def __gt__(self, other: "Polygon") -> bool:
        return self.edges == other.edges

In [2]:
# refactor Polygons sequence to be an iterable, evaluate properties lazily


class Polygons:
    def __init__(self, max_vertices: int, circumradious: float):
        if max_vertices <= 2 or circumradious <= 0:
            raise ValueError(
                "Polygons sequence must be initialized with max vertices greater than 2 and circumradious grater than 0."
            )
        
        self.max_vertices = max_vertices
        self.circumradious = circumradious
    
    @property
    def max_efficiency_polygon(self):
        polygons = list(iter(self))
        return sorted(polygons, key=lambda p: p.area / p.perimeter, reverse=True)[0]

    #def __getitem__(self, s: int | slice):
    #    return self.polygons[s]

    def __len__(self):
        return self.max_vertices - 2

    def __repr__(self):
        return f"{type(self).__name__}({self.max_vertices},{self.circumradious})"

    def __iter__(self):
        return self.PolygonsIter(self.max_vertices, self.circumradious) 


    class PolygonsIter:
        def __init__(self, max_vertices, circumradious):
            if max_vertices <= 2 or circumradious <= 0:
                raise ValueError(
                    "Polygons sequence must be initialized with max vertices greater than 2 and circumradious grater than 0."
                )
            self.max_vertices = max_vertices
            self.circumradious = circumradious
            self._i = 3

        def __iter__(self):
            return self

        def __next__(self):
            if self._i > self.max_vertices:
                raise StopIteration

            p = Polygon(self._i, self.circumradious)
            self._i += 1
            return p


In [3]:
pp = Polygons(10, 1)
list(pp)

[Polygon(3,1),
 Polygon(4,1),
 Polygon(5,1),
 Polygon(6,1),
 Polygon(7,1),
 Polygon(8,1),
 Polygon(9,1),
 Polygon(10,1)]

In [4]:
for p in Polygons(10, 1):
    print(p)

Polygon(3,1)
Polygon(4,1)
Polygon(5,1)
Polygon(6,1)
Polygon(7,1)
Polygon(8,1)
Polygon(9,1)
Polygon(10,1)


In [5]:
iter(Polygons(3,1))

In [6]:
max_ = Polygons(1_000_000,1).max_efficiency_polygon
print(max_.area)

3.141592653569122
